# Deploy Model to online endpoint

- Models created with MLFlow do not require a scoring script nor an environment, with this notebook, we will create a scopy.py file and use that with our model.

- Before running this notebook, run the **Chapter 6 Prep-Model Creation & Registration.ipynb** notebook to create and register a model for use

- Models created with MLFlow do not require a scoring script nor an environment

## In this notebook we will:

- Connect to your workspace.
- Create an online endpoint
- Retrieve and register a model from the job ran in the previous notebook
- Create a deployment
- Make an API Call to the managed online endpoint


In [53]:
import azure.ai.ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

print(azure.ai.ml._version.VERSION)

1.1.1


In [54]:
subscription_id = "b071bca8-0055-43f9-9ff8-ca9a144c2a6f"
resource_group = "aml-dev-rg"
workspace = "aml-dev"

In [55]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [56]:
#connect to the workspace
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/ML-Engineering-with-Azure-Machine-Learning-Service/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f6814daebf0>,
         subscription_id=b071bca8-0055-43f9-9ff8-ca9a144c2a6f,
         resource_group_name=aml-dev-rg,
         workspace_name=aml-dev)


### Configure endpoint

In [58]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "ch6-sdkv2-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"oneline endpoint": "titanic"},
)

## Create endpoint

Using the MLClient created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.


In [59]:
ml_client.begin_create_or_update(endpoint)

## Create deployment
A deployment is a set of resouces used for hosting the inferecing model using the *ManagedOnlineDeployment* class.  
Using the *ManagedOnlineDeployment* class, a developer can configure the following components

- name: name of the deployment
- endpoint_name: name of the endpoint to create the deployment under
- model: the model to use for the deployment
- instance_type: the VM side to use for deployment
- instance_count: the number of instances to use for the deployment
    

# Retrieve Model from registry

In [60]:
import mlflow

In [62]:
experiment = 'chapter6'
current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

df = mlflow.search_runs([experiment_id])


run_id = df['run_id'].iloc[-1]
print(run_id)
print(type(run_id))

e5ccf20f-54f7-4511-8ca7-15703445747c
sincere_shirt_95vn1fk2nc
<class 'str'>


In [63]:
print(run_id)
mlflow.set_experiment(experiment_name='Chapter6')
client = mlflow.tracking.MlflowClient()
client.list_artifacts(run_id=run_id)

sincere_shirt_95vn1fk2nc


[<FileInfo: file_size=-1, is_dir=False, path='estimator.html'>,
 <FileInfo: file_size=-1, is_dir=True, path='model'>,
 <FileInfo: file_size=-1, is_dir=True, path='system_logs'>,
 <FileInfo: file_size=-1, is_dir=False, path='training_confusion_matrix.png'>,
 <FileInfo: file_size=-1, is_dir=False, path='training_precision_recall_curve.png'>,
 <FileInfo: file_size=-1, is_dir=False, path='training_roc_curve.png'>,
 <FileInfo: file_size=-1, is_dir=True, path='user_logs'>]

In [64]:
import os
import shutil

file_path = client.download_artifacts(
    run_id, path="model"
)
shutil.copytree(file_path, './model', dirs_exist_ok=True)

/tmp/ipykernel_23870/3777299957.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  file_path = client.download_artifacts(


'./model'

In [65]:
model = Model(path="./model/model.pkl")

## Get Environment

In [72]:
env = ml_client.environments.get(name = 'job_base_env', version = "1")

## Creating Scoring Script

score.py file

In [73]:
import os

# Create a folder for the experiment files
script_folder = 'ManagedOnlineEndpoint'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

ManagedOnlineEndpoint folder created


In [74]:
%%writefile $script_folder/score.py

import os 
import json
import joblib
from pandas import json_normalize
import pandas as pd
import logging

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)
    logging.info("Init complete")

# Called when a request is received
def run(raw_data):
    dict= json.loads(raw_data)
    df = json_normalize(dict['raw_data']) 
    y_pred = model.predict(df)
    print(type(y_pred))
    
    result = {"result": y_pred.tolist()}
    return result

Overwriting ManagedOnlineEndpoint/score.py


## Configure the deployment

- retrieve the experiment id for this run, and the run id to retrieve the model from the registered model list

In [75]:
import datetime


blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment="azureml:job_base_env:16",
    code_configuration=CodeConfiguration(
        code="./ManagedOnlineEndpoint", scoring_script="score.py"
    ),
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

## Create deployment

In [76]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint ch6-sdkv2-endpt-11282324567965 exists
Uploading model.pkl (< 1 MB): 100%|██████████| 2.62k/2.62k [00:00<00:00, 62.5kB/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


........

In [77]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating..will take about 10 minutes to deploy...')
    time.sleep(5)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 minutes to deploy...
..Updating..will take about 10 minutes to deploy...
.Updating..will take about 10 m

'Succeeded'

.

In [78]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

In [79]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating')
    time.sleep(5)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

'Succeeded'

In [80]:
## Get Endpoint details

# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint)
print(' ')
# existing traffic details
print(endpoint.traffic)
print(' ')
# Get the scoring URI
print('uri: ' + str(endpoint.scoring_uri))
primary_key = ml_client.online_endpoints.get_keys(name = online_endpoint_name).primary_key
print(' ')
print('primary key: ' + str(primary_key))

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://ch6-sdkv2-endpt-11282324567965.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://ch6-sdkv2-endpt-11282324567965.eastus2.inference.ml.azure.com/swagger.json', 'name': 'ch6-sdkv2-endpt-11282324567965', 'description': 'titanic online endpoint for mlflow model', 'tags': {'oneline endpoint': 'titanic'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/b071bca8-0055-43f9-9ff8-ca9a144c2a6f/resourcegroups/aml-dev-rg/providers/microsoft.machinelearningservices/workspaces/aml-dev/onlineendpoints/ch6-sdkv2-endpt-11282324567965', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/b071bca8-0055-43f9-9ff8-ca9a144c2a6f/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:9634b616-29cb-4345-ae22-4be4e4bfe009:e1178544-8fe9-4b48-bf87-eb94f71c0b34?api-version=2022-02-01-preview'}, 'id': '/subscriptions/b071bca8-0055

In [81]:
import mltable
registered_v1_data_asset = ml_client.data.get(name='titanic_prepped_mltable_x2', version='1')
print(registered_v1_data_asset.path)

tbl = mltable.load(uri=registered_v1_data_asset.path)
df = tbl.to_pandas_dataframe()

columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]
X_raw


azureml://subscriptions/b071bca8-0055-43f9-9ff8-ca9a144c2a6f/resourcegroups/aml-dev-rg/workspaces/aml-dev/datastores/workspaceblobstore/paths/LocalUpload/b3e9d2d76d36b52fc88b17546f0f0460/titanic_prepped_mltable/


Unrecognized authentication type: None
Unrecognized authentication type: None


,Embarked,Loc,Sex,Pclass,Age,Fare,GroupSize
0,S,X,m,3,22.0,7.25,2
1,C,C,f,1,38.0,71.2833,2
2,S,X,f,3,26.0,7.925,1
3,S,C,f,1,35.0,53.1,2
4,S,X,m,3,35.0,8.05,1
...,...,...,...,...,...,...,...
886,S,X,m,2,27.0,13.0,1
887,S,B,f,1,19.0,30.0,1
888,S,X,f,3,21.5,23.45,4
889,C,C,m,1,26.0,30.0,1


In [84]:
import json
url = endpoint.scoring_uri
api_key = primary_key  # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
import requests

def MakePrediction(df):
    endpoint_url = url
    body = df.to_json(orient='records') 
    body = '{"raw_data": ' + body + '}'
    print(body)
    r = requests.post(endpoint_url, headers=headers, data=body)
    return (r.json())


columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]


dftest = X_raw.head(5)

results = MakePrediction(dftest)

val = results['result']
print('')
print('predictions')
print(val)

{"raw_data": [{"Embarked":"S","Loc":"X","Sex":"m","Pclass":"3","Age":"22.0","Fare":"7.25","GroupSize":"2"},{"Embarked":"C","Loc":"C","Sex":"f","Pclass":"1","Age":"38.0","Fare":"71.2833","GroupSize":"2"},{"Embarked":"S","Loc":"X","Sex":"f","Pclass":"3","Age":"26.0","Fare":"7.925","GroupSize":"1"},{"Embarked":"S","Loc":"C","Sex":"f","Pclass":"1","Age":"35.0","Fare":"53.1","GroupSize":"2"},{"Embarked":"S","Loc":"X","Sex":"m","Pclass":"3","Age":"35.0","Fare":"8.05","GroupSize":"1"}]}

predictions
[0, 1, 0, 1, 0]
